# Example p-MP Optimisation

## Getting Data

In [5]:
using OpenStreetMapX
using JLD
using CSV
using DataFrames
using StatsBase
using JuMP
using Gurobi

include("../src/data/osm.jl")
include("../src/data/distance_matrix.jl")
include("../src/optimisation/mip/p_mp_model.jl")
include("../src/optimisation/mip/mclp_model.jl")
include("../src/optimisation/metrics.jl")
include("../src/optimisation/iterative/optimisation.jl")

location_optimization_radius (generic function with 5 methods)

In [6]:
LOCATION = "winnipeg"
DIR_PATH = "../osm_maps/" * LOCATION * "/"
OSM_PATH = DIR_PATH * "map.osm"

isdir(DIR_PATH) ? nothing : mkdir(DIR_PATH)
isfile(OSM_PATH) ? nothing : get_osm_by_bbox(-97.3581, 49.7147, -96.9475, 49.9954, OSM_PATH)

In [7]:
mx = get_map_data(OSM_PATH, use_cache=true, road_levels=Set(1:5), trim_to_connected_graph = true)

┌ Info: Read map data from cache ../osm_maps/winnipeg\map.osm.cache
└ @ OpenStreetMapX C:\Users\Kiryl\.julia\packages\OpenStreetMapX\Yo58L\src\parseMap.jl:93


MapData(OpenStreetMapX.Bounds{LLA}(49.7147, 49.9954, -97.3581, -96.9475), Dict{Int64, ENU}(7799632408 => ENU(1494.130907217481, 4152.141303999004, -1.5273067814109709), 285736360 => ENU(1101.5882285733383, -15065.239259369318, -17.90216221123228), 1488823329 => ENU(7897.341566968542, -2778.649740685668, -5.485410342386103), 741689169 => ENU(-12734.403532880367, -9770.036591763856, -20.17696398425869), 256153327 => ENU(-7823.9314265355515, 4328.561832938522, -6.259356765273878), 631381980 => ENU(-1886.773140262505, 12270.444491945469, -12.091470101295272), 2675696260 => ENU(4681.149026966116, 8111.047167332585, -6.876158216364274), 136147026 => ENU(5277.55875406265, 3364.7416652434094, -3.067432797258334), 1227198431 => ENU(2756.332940129022, 6635.241107579223, -4.048647734779024), 1112293467 => ENU(1331.3070969172638, -797.750121809354, -0.18860118046427488)…), OpenStreetMapX.Way[OpenStreetMapX.Way(4443871, [27286480, 27286484, 34040347, 27286488, 27366907, 27286482, 27366892, 34040311

## Processing Data

In [8]:
TREES_PATH = DIR_PATH * "trees.jld"
METRIC = "time"

if isfile(TREES_PATH)
    node_data = load(TREES_PATH, "node_data")
else
    node_data = create_distance_trees(mx, METRIC)
    save(TREES_PATH, "node_data", node_data)
end

Dict{Int64, Tuple{Vector{Int64}, Vector{Float64}}} with 5752 entries:
  2428190804 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  128583108  => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  285736360  => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  6598934129 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  741689169  => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  2709909467 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  1658443102 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  1115858543 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  278613704  => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  4785952268 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  3131822521 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  1512335051 => ([2428190804, 128583108

In [9]:
DISTANCE_MATRIX_PATH = DIR_PATH * "distance_matrix.jld"
NODE_IDS_PATH = DIR_PATH * "node_ids.jld"


if isfile(DISTANCE_MATRIX_PATH) & isfile(NODE_IDS_PATH)
    all_node_ids = load(NODE_IDS_PATH, "all_node_ids")
    distance_matrix = load(DISTANCE_MATRIX_PATH, "distance_matrix")
else
    all_node_ids = collect(keys(node_data))    
    distance_matrix = build_distance_matrix_graph(all_node_ids, all_node_ids, node_data)
    save(DISTANCE_MATRIX_PATH, "distance_matrix", distance_matrix)
    save(NODE_IDS_PATH, "all_node_ids", all_node_ids)
end

5752×5752 Matrix{Int64}:
    0  387  1028   389  1154   588  …   797   787   517  723   830   904
  418    0   713   539   860   517      669   660   130  429   466   777
 1048  770     0  1130   796  1087     1066  1138   682  620   549  1254
  396  549  1133     0  1184   552      770   760   670  754   889   877
 1153  917   764  1181     0  1010      702   791   850  457   608   674
  567  543  1097   561  1011     0  …   553   543   646  581   732   660
  563  549  1103   269  1154   288      695   685   652  724   859   802
  767  674  1071   768   739   568       85   214   750  482   633   285
  672  579   894   673   735   460      170   299   608  305   456   371
  727  415   353   775   592   732      710   782   327  265   291   899
 1135  871   613  1231   424  1096  …   993  1065   783  547   461  1019
  783  471   344   831   583   789      767   839   383  321   335   956
  622  578   854   983  1079  1040     1193  1184   592  827   831  1300
    ⋮                     

## Model

### Iterative

In [10]:
P = 9
R = 300.0
RUIN_RANDOM = 0.75
N_GLOBAL_RANDOM_TRIES = 100

initial_nodes = get_best_of_n_random_locations(mx, P, N_GLOBAL_RANDOM_TRIES)
t = @elapsed opt_loc = location_optimization_radius(mx, initial_nodes, 500, "time", RUIN_RANDOM, R)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
Found local minimum. Stopping.


73.2620643

In [11]:
println("Final nodes: ", opt_loc[1][end])
println("Final objective: ", opt_loc[2][end])
println("Final time: ", t)

Final nodes: [82772408, 4688961724, 369118765, 1197819038, 735138811, 128686681, 370073744, 469531931, 367838436]
Final objective: 165.5865064891183
Final time: 73.2620643


### Hierarchical-Iterative

In [12]:
mx_3 = get_map_data(OSM_PATH, use_cache=false, road_levels=Set(1:3), trim_to_connected_graph = true);
mx_4 = get_map_data(OSM_PATH, use_cache=false, road_levels=Set(1:4), trim_to_connected_graph = true);
mx_5 = get_map_data(OSM_PATH, use_cache=false, road_levels=Set(1:5), trim_to_connected_graph = true);

In [13]:
P = 9
R = 300.0
RUIN_RANDOM = 0.75
N_GLOBAL_RANDOM_TRIES = 100

initial_nodes = get_best_of_n_random_locations(mx_3, P, N_GLOBAL_RANDOM_TRIES)

t_1 = @elapsed opt_loc_1 = location_optimization_radius(mx_3, initial_nodes, 500, "time", RUIN_RANDOM, R)
t_2 = @elapsed opt_loc_2 = location_optimization_radius(mx_4, opt_loc_1[1][end], 500, "time", RUIN_RANDOM, R)
t_3 = @elapsed opt_loc_3 = location_optimization_radius(mx_5, opt_loc_2[1][end], 500, "time", RUIN_RANDOM, R)

t = t_1 + t_2 + t_3

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
Found local minimum. Stopping.
1
2
3
4
5
6
7
8
9
10
11
12
13
Found local minimum. Stopping.
1
2
3
4
5
6
7
8
9
10
11
12
13
Found local minimum. Stopping.


106.4919915

In [14]:
println("Final nodes: ", opt_loc_3[1][end])
println("Final objective: ", opt_loc_3[2][end])
println("Final time: ", t)

Final nodes: [3762127594, 128686681, 469531931, 735138811, 370073744, 82772408, 310249325, 1197820121, 369294136]
Final objective: 164.87366812704377
Final time: 106.4919915


In [15]:
println(opt_loc_1[2][end])
println(opt_loc_2[2][end])
println(opt_loc_3[2][end])

185.4431385223837
189.0154379200478
164.87366812704377


### p-MP

#### Sampling

In [16]:
P = 9
M = 1000
N = 1000
Q = 1

m_idx = sample(1:length(all_node_ids), M, replace=false)
n_idx = sample(1:length(all_node_ids), N, replace=false)

m = all_node_ids[m_idx]
n = all_node_ids[n_idx]
h = repeat([1], N)

sample_distance_matrix = distance_matrix[m_idx, n_idx]

1000×1000 Matrix{Int64}:
  458   636   316   504  1129  528  …   546   512   561   741   313   706
  776   881   221   749  1358  756      774   754   803   970   541  1028
  543   489   551   358  1057  478      323    97   145   692   264   782
  765   278   489   147  1274  699      514   237   206   914   486  1004
  597   591   673   460   978  498      162   279   250   743   315   833
  838   498   946   545   882  678  …   375   372   342   968   540  1058
  639   739    99   607  1215  613      632   612   660   827   399   886
  888   254   821   301  1044  823      572   274   243  1037   609  1127
  718  1051   888   935   557  175      432   702   726   501   406   663
 1034   729  1141   776   819  625      457   630   600   953   735  1115
  748   655   855   523   961  521  …   227   343   314   849   449   967
  765   308   698   286  1138  699      449   150   119   914   486  1004
  841   420   909   467   972  709      390   318   288   971   542  1060
    ⋮        

In [ ]:
results = optimise_p_mp_model(
    sample_distance_matrix, P, m, n, h, Q
)

____ [2022-04-03T21:30:28.833] - Defining a JuMP Model


In [ ]:
results

In [ ]:
println("Final nodes: ", results["final_nodes"])
println("Final objective: ", get_objective(results["final_nodes"], mx))
println("Final time: ", results["t"])

#### Aggregating

In [ ]:
AGG_LOCATION = "winnipeg_agg_9"
AGG_DIR_PATH = "../osm_maps/" * AGG_LOCATION * "/"
AGG_OSM_PATH = AGG_DIR_PATH * "map.osm"
AGG_DISTANCE_MATRIX_PATH = AGG_DIR_PATH * "distance_matrix.jld"

agg_distance_matrix = load(AGG_DISTANCE_MATRIX_PATH, "distance_matrix")

In [ ]:
P = 9
M = size(agg_distance_matrix)[1]
N = size(agg_distance_matrix)[1]
Q = 1

In [ ]:
results = optimise_p_mp_model(
    agg_distance_matrix, P, m, n, h, Q
)

In [ ]:
println("Final nodes: ", results["final_nodes"])
println("Final objective: ", get_objective(results["final_nodes"], mx))
println("Final time: ", results["t"])

### MCLP

#### Sampling

In [ ]:
P = 9
M = 1000
N = 1000
Q = 1
R = 300.0

m_idx = sample(1:length(all_node_ids), M, replace=false)
n_idx = sample(1:length(all_node_ids), N, replace=false)

m = all_node_ids[m_idx]
n = all_node_ids[n_idx]
h = repeat([1], N)

sample_distance_matrix = distance_matrix[m_idx, n_idx]

In [ ]:
results = optimise_mclp_model(sample_distance_matrix, P, m, n, h, Q, R)

In [ ]:
println("Final nodes: ", results["final_nodes"])
println("Final objective: ", get_objective(results["final_nodes"], mx))
println("Final time: ", results["t"])

#### Aggregating